In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

def fix_code(x,l): # Función que agrega 0's por delante en caso de su longitud sea menor a "l"
  x = str(x)
  while len(x) < l:
    x = '0'+x
  return x

def perc(x,val_qua): # Función que reemplaza los valores que son mayores al percentil
  while x > val_qua:
    x = float(val_qua)
  return x

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
HT_PATH = '/content/drive/MyDrive/Hackaton BBVA'
VR_PATH = '/content/drive/MyDrive/Reto Vinculación rentable'
TC_USD_SOL = 3.97

In [3]:
tablas={}
for i in range(1,8):
  base = '/base 0'+str(i)+'_V3.csv'
  try:
    tablas[i] = pd.read_csv(VR_PATH+base, sep = '|')
  except:
    tablas[i] = pd.read_csv(VR_PATH+'/base 07_V3 (1).csv', sep = '|')

In [4]:
# Limpieza 1_SOCIODEMOGRAFICOS
df_socio = tablas[1].drop(['DEPENDIENTES', 'SITUACION_LABORAL', 'TIPO_EMPLEO', 'TIPO_PROFESION'], 1)
df_socio.CODIGO_CLIENTE = df_socio.CODIGO_CLIENTE.astype(str)
df_socio.CODIGO_CLIENTE = df_socio.CODIGO_CLIENTE.apply(lambda x: fix_code(x,8))
df_socio.EDAD = df_socio.EDAD.fillna(round((df_socio.EDAD.quantile(0.25)+df_socio.EDAD.quantile(0.75))/2))
df_socio.EDAD = df_socio.EDAD.astype(int)
df_socio.GENERO = df_socio.GENERO.fillna('X')
df_socio.UBIGEO_DIRECCION = df_socio.UBIGEO_DIRECCION.fillna(0).astype(int)
df_socio.ESTADO_CIVIL = np.where(df_socio.ESTADO_CIVIL.isin(['S', 'V', 'D', 'C']), df_socio.ESTADO_CIVIL, 'O')
df_socio.INGRESO_MENSUAL = df_socio.INGRESO_MENSUAL.fillna(0)

In [5]:
# Limpieza 2_ADICIONAL
df_adicional = tablas[2].drop(['LINEA VIGENTE', 'SALDO MEDIO', 'RIESGO3'], 1)
df_adicional.CODIGO_CLIENTE = df_adicional.CODIGO_CLIENTE.astype(str)
df_adicional.CODIGO_CLIENTE = df_adicional.CODIGO_CLIENTE.apply(lambda x: fix_code(x,8))
df_adicional = df_adicional.drop_duplicates()

In [6]:
# Limpieza 3_TENENCIA_PRODUCTOS
df_tenencia = tablas[3].drop(['TIPO_DOCUMENTO', 'NUMERO_DOCUMENTO'],1)
df_tenencia.CODIGO_CLIENTE = df_tenencia.CODIGO_CLIENTE.astype(str)
df_tenencia.CODIGO_CLIENTE = df_tenencia.CODIGO_CLIENTE.apply(lambda x: fix_code(x,8))
df_tenencia = df_tenencia.drop_duplicates()
df_tenencia['FAM_GLOBAL'] = df_tenencia[['FAM_COBRANZAS', 'FAM_COMEX', 'FAM_PAGOHABERES', 'FAM_PROVEEDORES', 'FAM_SUPPLY', 'FAM_VISANET']].max(axis=1)
df_tenencia = df_tenencia.drop(['FAM_COBRANZAS', 'FAM_COMEX', 'FAM_PAGOHABERES', 'FAM_PROVEEDORES', 'FAM_SUPPLY', 'FAM_VISANET'],1)

In [7]:
# Limpieza 4_RENTABILIDAD_CLIENTE
df_rent_cliente = tablas[4].drop(['RATIO_RENTABILIDAD', 'MARGEN_FINANCIERO', 'COMISIONES'],1)
df_rent_cliente.CODIGO_CLIENTE = df_rent_cliente.CODIGO_CLIENTE.astype(str)
df_rent_cliente.CODIGO_CLIENTE = df_rent_cliente.CODIGO_CLIENTE.apply(lambda x: fix_code(x,8))
df_rent_cliente = df_rent_cliente.drop_duplicates()

In [8]:
# Limpieza 5_RENTABILIDAD_PRODUCTO
df_rent_prod = tablas[5].drop(['RATIO_RENTABILIDAD', 'MARGEN_FINANCIERO', 'COMISIONES'],1)
df_rent_prod.CODIGO_CLIENTE = df_rent_prod.CODIGO_CLIENTE.astype(str)
df_rent_prod.CODIGO_CLIENTE = df_rent_prod.CODIGO_CLIENTE.apply(lambda x: fix_code(x,8))
df_rent_prod.loc[df_rent_prod['DIVISA'] == 2, 'SALDO_PUNTUAL'] = df_rent_prod['SALDO_PUNTUAL'] * TC_USD_SOL
df_rent_prod.loc[df_rent_prod['DIVISA'] == 2, 'MARGEN_BRUTO'] = df_rent_prod['MARGEN_BRUTO'] * TC_USD_SOL
df_rent_prod.loc[df_rent_prod['DIVISA'] == 2, 'EXPOSICION_DEFAULT'] = df_rent_prod['EXPOSICION_DEFAULT'] * TC_USD_SOL
df_rent_prod.loc[df_rent_prod['DIVISA'] == 2, 'PERDIDA_ESPERADA'] = df_rent_prod['PERDIDA_ESPERADA'] * TC_USD_SOL
df_rent_prod.loc[df_rent_prod['DIVISA'] == 2, 'NUMERADOR_RATIO_RENTABILIDAD'] = df_rent_prod['NUMERADOR_RATIO_RENTABILIDAD'] * TC_USD_SOL
df_rent_prod.loc[df_rent_prod['DIVISA'] == 2, 'DENOMINADOR_RATIO_RENTABILIDAD'] = df_rent_prod['DENOMINADOR_RATIO_RENTABILIDAD'] * TC_USD_SOL
df_rent_prod = df_rent_prod.drop('DIVISA',1)
df_rent_prod = df_rent_prod.groupby(['CODIGO_CLIENTE', 'PRODUCTO', 'PERIODO']).sum().reset_index()

In [9]:
# Limpieza 6_MOVIMIENTO_TARJETA
df_movimiento_tarj = tablas[6]
df_movimiento_tarj = df_movimiento_tarj.drop(['DETALLE_TRANSACCION1', 'DETALLE_TRANSACCION2', 'DETALLE_TRANSACCION3'],1)
df_movs = df_movimiento_tarj.groupby('TIPO_TARJETA')['TIPO_TARJETA'].count().sort_values(ascending=False)
moda_tipo_tarjeta = df_movs.index[0]
df_movimiento_tarj.TIPO_TARJETA = df_movimiento_tarj.TIPO_TARJETA.fillna(moda_tipo_tarjeta)
df_movimiento_tarj.TIPO_TARJETA = df_movimiento_tarj.TIPO_TARJETA.astype(str)
df_movimiento_tarj.CODIGO_CLIENTE = df_movimiento_tarj.CODIGO_CLIENTE.astype(str)
df_movimiento_tarj.CODIGO_CLIENTE = df_movimiento_tarj.CODIGO_CLIENTE.apply(lambda x: fix_code(x,8))
df_movimiento_tarj.TIPO_TRANSACCION = df_movimiento_tarj.TIPO_TRANSACCION.astype(str)
df_movimiento_tarj.TIPO_TRANSACCION = df_movimiento_tarj.TIPO_TRANSACCION.apply(lambda x: fix_code(x,2))
df_movimiento_tarj = df_movimiento_tarj.groupby(['CODIGO_CLIENTE', 'TIPO_TRANSACCION', 'TIPO_TARJETA', 'PERIODO']).sum().reset_index()

In [10]:
# Limpieza 7_DEUDA_SSFF
# Número de documentos NULOS a ceros
df_deuda_ssff = tablas[7].drop(['PRODUCTO', 'TIPO_MOVIMIENTO'],1)
df_deuda_ssff.ENTIDAD = df_deuda_ssff.ENTIDAD.astype(str)
df_deuda_ssff.NUMERO_DOCUMENTO = df_deuda_ssff.NUMERO_DOCUMENTO.fillna(0)
df_deuda_ssff.NUMERO_DOCUMENTO = df_deuda_ssff.NUMERO_DOCUMENTO.astype(int)
df_deuda_ssff.CODIGO_CLIENTE = df_deuda_ssff.CODIGO_CLIENTE.astype(str)
df_deuda_ssff.CODIGO_CLIENTE = df_deuda_ssff.CODIGO_CLIENTE.apply(lambda x: fix_code(x,8))
df_deuda_ssff.loc[df_deuda_ssff['DIVISA'] == 2, 'SALDO'] = df_deuda_ssff['SALDO'] * TC_USD_SOL
df_deuda_ssff = df_deuda_ssff.drop('DIVISA',1)
df_deuda_ssff = df_deuda_ssff.reset_index()

In [11]:
# Inicio de Cruces
df_master = df_socio.merge(df_adicional, on=['PERIODO','CODIGO_CLIENTE'], how='inner', suffixes=(None, '_x'))
df_master = df_master.merge(df_tenencia, on=['PERIODO','CODIGO_CLIENTE'], how='inner')
df_master = df_master.merge(df_rent_cliente, on=['PERIODO','CODIGO_CLIENTE'], how='inner')
df_master = df_master.drop(['TIPO_DOCUMENTO_x', 'NUMERO_DOCUMENTO_x'],1)

In [12]:
# Enriquecemos con ABONOS y COBROS para TDC
df_mov_tdc = df_movimiento_tarj
df_mov_tdc.TIPO_TRANSACCION = df_mov_tdc.TIPO_TRANSACCION.replace(['00', '01', '02', '36', '60', '66', '77', '89', '90', '95'], 'CARGOS_TDC')
df_mov_tdc.TIPO_TRANSACCION = df_mov_tdc.TIPO_TRANSACCION.replace(['20', '22', '27', '79', '87'], 'ABONOS_TDC')
df_mov_tdc = df_mov_tdc[(df_mov_tdc.TIPO_TRANSACCION == 'CARGOS_TDC') | (df_mov_tdc.TIPO_TRANSACCION == 'ABONOS_TDC')]
df_mov_tdc = df_mov_tdc.groupby(['CODIGO_CLIENTE', 'PERIODO', 'TIPO_TRANSACCION']).sum().reset_index()

df_mov_tdc = (df_mov_tdc
 .pivot(index=['CODIGO_CLIENTE','PERIODO'],
        columns='TIPO_TRANSACCION',
        values='IMPORTE_TRANSACCION'
        )
 .reset_index()
 .rename_axis(None, axis="columns")
 .fillna(0)
 )

df_master = df_master.merge(df_mov_tdc, on=['PERIODO','CODIGO_CLIENTE'], how='left')
df_master.ABONOS_TDC = df_master.ABONOS_TDC.fillna(0)
df_master.CARGOS_TDC = df_master.CARGOS_TDC.fillna(0)

In [13]:
# Enriquecemos con información de PRODUCTOS
df_rent_prod_tot = df_rent_prod.groupby(['CODIGO_CLIENTE', 'PERIODO']).sum().reset_index()
df_rent_prod_tot = df_rent_prod_tot.add_suffix('_RENT_PROD')

df_rent_prod_prod = (df_rent_prod
 .pivot(index=['CODIGO_CLIENTE','PERIODO'],  # queremos conservar esta columna como columna
        columns='PRODUCTO', # es la columna que queremos pivotear
        values='PRODUCTO' # son los valores con los que vamos a poblar la columna pivoteada
        )
 .reset_index()
 .rename_axis(None, axis="columns")
 .fillna(0)
 )

df_rent_prod_prod.loc[df_rent_prod_prod['AUTOS']=='AUTOS',['AUTOS']] = 1
df_rent_prod_prod.loc[df_rent_prod_prod['CARTERA']=='CARTERA',['CARTERA']] = 1
df_rent_prod_prod.loc[df_rent_prod_prod['COMEXT']=='COMEXT',['COMEXT']] = 1
df_rent_prod_prod.loc[df_rent_prod_prod['CONSUMO']=='CONSUMO',['CONSUMO']] = 1
df_rent_prod_prod.loc[df_rent_prod_prod['HIPOTECARIO']=='HIPOTECARIO',['HIPOTECARIO']] = 1
df_rent_prod_prod.loc[df_rent_prod_prod['LEASING']=='LEASING',['LEASING']] = 1
df_rent_prod_prod.loc[df_rent_prod_prod['PRESTAMOS_COMERCIALES']=='PRESTAMOS_COMERCIALES',['PRESTAMOS_COMERCIALES']] = 1
df_rent_prod_prod.loc[df_rent_prod_prod['TARJETAS']=='TARJETAS',['TARJETAS']] = 1
df_rent_prod_prod.loc[df_rent_prod_prod['TJ_EMPRESAS']=='TJ_EMPRESAS',['TJ_EMPRESAS']] = 1
df_rent_prod_prod['NO_PRODUCTOS'] = df_rent_prod_prod['AUTOS'] + df_rent_prod_prod['CARTERA'] + df_rent_prod_prod['COMEXT'] + df_rent_prod_prod['CONSUMO'] + df_rent_prod_prod['HIPOTECARIO'] + df_rent_prod_prod['LEASING'] + df_rent_prod_prod['PRESTAMOS_COMERCIALES'] + df_rent_prod_prod['TARJETAS'] + df_rent_prod_prod['TJ_EMPRESAS']

df_rent_prod_full = df_rent_prod_prod.merge(df_rent_prod_tot, left_on=['CODIGO_CLIENTE', 'PERIODO'], right_on=['CODIGO_CLIENTE_RENT_PROD', 'PERIODO_RENT_PROD'], how='inner')
df_rent_prod_full = df_rent_prod_full.drop(['CODIGO_CLIENTE_RENT_PROD', 'PERIODO_RENT_PROD'],1)

df_rent_prod_full = df_rent_prod_full.add_suffix('_RENT_PROD')
df_rent_prod_full = df_rent_prod_full.rename(columns={'CODIGO_CLIENTE_RENT_PROD':'CODIGO_CLIENTE',
                                                      'PERIODO_RENT_PROD':'PERIODO'})

df_master = df_master.merge(df_rent_prod_full, on=['CODIGO_CLIENTE', 'PERIODO'], how='left')
df_master = df_master.fillna(0)

In [14]:
# division de clientes y no clientes
# df_master
# tomamos los clientes unicos que tienen entidad bbva
clientes_bbva = df_deuda_ssff.loc[df_deuda_ssff['ENTIDAD']=='BBVA']
clientes_bbva_uniq = df_socio[['CODIGO_CLIENTE']].merge(clientes_bbva[['CODIGO_CLIENTE']], on=['CODIGO_CLIENTE'], how='inner')
clientes_bbva_uniq = clientes_bbva_uniq.groupby('CODIGO_CLIENTE').count()
list_clientes_bbva = clientes_bbva_uniq.reset_index()['CODIGO_CLIENTE'].tolist()

# tomamos los clientes unicos que tienen entidad otro
clientes_otro = df_deuda_ssff.loc[df_deuda_ssff['ENTIDAD']=='OTRO']
clientes_otro_uniq = df_socio[['CODIGO_CLIENTE']].merge(clientes_otro[['CODIGO_CLIENTE']], on=['CODIGO_CLIENTE'], how='inner')
clientes_otro_uniq = clientes_otro_uniq.groupby('CODIGO_CLIENTE').count()

# separamos los clientes que tienen entidad otro y no tienen entidad bbva
cl_aux = pd.merge(clientes_bbva_uniq, clientes_otro_uniq, on=['CODIGO_CLIENTE'], how='outer', indicator=True)
cl_separados = cl_aux[cl_aux['_merge'] == 'right_only']
cl_separados = cl_separados.groupby('CODIGO_CLIENTE').count()
list_cl_separados = cl_separados.reset_index()['CODIGO_CLIENTE'].tolist()


In [15]:
df_deuda_ssff_sinBBVA = df_deuda_ssff.loc[df_deuda_ssff['ENTIDAD']!='BBVA']
df_deuda_ssff_agg = (df_deuda_ssff_sinBBVA.groupby(['CODIGO_CLIENTE', 'PERIODO','DETALLE_PRODUCTO'])
                      .agg({'SALDO':sum}).reset_index()) #,'DIAS_ATRASO':sum

df_deuda_ssff_piv = (df_deuda_ssff_agg
 .pivot(index=['CODIGO_CLIENTE', 'PERIODO'],
        columns='DETALLE_PRODUCTO',
        values='DETALLE_PRODUCTO')
 .reset_index()
 .rename_axis(None, axis="columns")
 .fillna(0)
 )

df_deuda_ssff_piv.loc[df_deuda_ssff_piv['AUTOS']=='AUTOS',['AUTOS']] = 1
df_deuda_ssff_piv.loc[df_deuda_ssff_piv['CARTERA']=='CARTERA',['CARTERA']] = 1
df_deuda_ssff_piv.loc[df_deuda_ssff_piv['COMEXT']=='COMEXT',['COMEXT']] = 1
df_deuda_ssff_piv.loc[df_deuda_ssff_piv['CONSUMO']=='CONSUMO',['CONSUMO']] = 1
df_deuda_ssff_piv.loc[df_deuda_ssff_piv['HIPOTECARIO']=='HIPOTECARIO',['HIPOTECARIO']] = 1
df_deuda_ssff_piv.loc[df_deuda_ssff_piv['LEASING']=='LEASING',['LEASING']] = 1
df_deuda_ssff_piv.loc[df_deuda_ssff_piv['PRESTAMOS_COMERCIALES']=='PRESTAMOS_COMERCIALES',['PRESTAMOS_COMERCIALES']] = 1
df_deuda_ssff_piv.loc[df_deuda_ssff_piv['TARJETAS']=='TARJETAS',['TARJETAS']] = 1
df_deuda_ssff_piv.loc[df_deuda_ssff_piv['TJ_EMPRESAS']=='TJ_EMPRESAS',['TJ_EMPRESAS']] = 1

df_deuda_ssff_piv['NO_PRODUCTOS'] = df_deuda_ssff_piv['AUTOS'] + df_deuda_ssff_piv['CARTERA'] + df_deuda_ssff_piv['COMEXT'] + df_deuda_ssff_piv['CONSUMO'] + df_deuda_ssff_piv['HIPOTECARIO'] + df_deuda_ssff_piv['LEASING'] + df_deuda_ssff_piv['PRESTAMOS_COMERCIALES'] + df_deuda_ssff_piv['TARJETAS'] + df_deuda_ssff_piv['TJ_EMPRESAS']

df_deuda_ssff_agg = df_deuda_ssff_agg.drop(['DETALLE_PRODUCTO'],axis=1)
df_deuda_ssff_piv_agg = df_deuda_ssff_piv.merge(df_deuda_ssff_agg, on=['PERIODO','CODIGO_CLIENTE'], how='left')

df_deuda_ssff_piv_agg = df_deuda_ssff_piv_agg.add_suffix('_OTROS')
df_deuda_ssff_piv_agg = df_deuda_ssff_piv_agg.rename(columns={'CODIGO_CLIENTE_OTROS':'CODIGO_CLIENTE',
                                                      'PERIODO_OTROS':'PERIODO'})

df_master = df_master.merge(df_deuda_ssff_piv_agg, on=['PERIODO','CODIGO_CLIENTE'], how='left').fillna(0)

In [16]:
pd_update_att=pd.read_csv(HT_PATH+'/Tablas_guardadas/Tiempo_de_vida.csv')
cruce_abandono = pd_update_att[['CODIGO_CLIENTE','att','abandono','PERIODO']].groupby(['CODIGO_CLIENTE','PERIODO']).mean().reset_index()
cruce_abandono.CODIGO_CLIENTE = cruce_abandono.CODIGO_CLIENTE.astype(str)
cruce_abandono.CODIGO_CLIENTE = cruce_abandono.CODIGO_CLIENTE.apply(lambda x: fix_code(x,8))
cruce_abandono = cruce_abandono.rename(columns={'att':'ATT',
                                                      'abandono':'ABANDONO'})

In [17]:
df_master = df_master.merge(cruce_abandono, on=['PERIODO','CODIGO_CLIENTE'], how='left').fillna(0)

In [18]:
cols_perc9 = ['INGRESO_MENSUAL','SALDO_MEDIO_VISTA','SALDO_MEDIO_AHORRO','SALDO_MEDIO_CTS','SALDO_MEDIO_PLAZO',
              'SALDO_MEDIO_FONDO_MUTUO','SALDO_MEDIO_AUTOS','SALDO_MEDIO_CONSUMO','SALDO_MEDIO_TARJETAS',
              'SALDO_MEDIO_HIPOTECARIO','SALDO_MEDIO_CARTERA','SALDO_MEDIO_LEASING','SALDO_MEDIO_PRESTAMOS_COMERCIALES',
              'SALDO_MEDIO_COMEXT','SALDO_MEDIO_TJ_EMPRESAS','SALDO_MEDIO_INVERSION_RENTABLE','SALDO_MEDIO_CARTERA_ATRASADA',
              'ABONOS_TDC','CARGOS_TDC']

for i in cols_perc9:
  val_qua = df_master.loc[:,i].quantile(.95)
  if val_qua == 0:
    val_qua = df_master.loc[:,i].quantile(.99)
  if val_qua == 0:
    val_qua = df_master.loc[:,i].quantile(.995)
  if val_qua == 0:
    val_qua = df_master.loc[:,i].quantile(.999)
  if val_qua == 0:
    val_qua = df_master.loc[:,i].quantile(.9995)
  if val_qua == 0:
    val_qua = df_master.loc[:,i].quantile(.9999)
  if val_qua == 0:
    val_qua = df_master.loc[:,i].quantile(.99999)
  
  df_master.loc[:,i] = df_master.loc[:,i].apply(lambda x: perc(x,val_qua))


In [19]:
# filtramos para obtener df separado con entidad bbva y otro o solo bbva
df_bbva = df_master.loc[~df_master['CODIGO_CLIENTE'].isin(list_cl_separados)]
# filtramos para obtener df separado con solo entidad otro sin bbva
df_otros = df_master.loc[df_master['CODIGO_CLIENTE'].isin(list_cl_separados)]

In [20]:
for i in df_master.columns:
  print("Columna "+i+": "+df_master[i].isnull().sum().astype(str))

Columna CODIGO_CLIENTE: 0
Columna TIPO_DOCUMENTO: 0
Columna NUMERO_DOCUMENTO: 0
Columna EDAD: 0
Columna GENERO: 0
Columna UBIGEO_DIRECCION: 0
Columna SEGMENTO_COMERCIAL: 0
Columna ESTADO_CIVIL: 0
Columna AFILIACION_SMS: 0
Columna AFILIACION_BANCA_ONLINE: 0
Columna INGRESO_MENSUAL: 0
Columna GRUPO_RIESGO: 0
Columna PERIODO: 0
Columna RIESGO1: 0
Columna RIESGO2: 0
Columna SALDO_MEDIO_VISTA: 0
Columna SALDO_MEDIO_AHORRO: 0
Columna SALDO_MEDIO_CTS: 0
Columna SALDO_MEDIO_PLAZO: 0
Columna SALDO_MEDIO_FONDO_MUTUO: 0
Columna SALDO_MEDIO_AUTOS: 0
Columna SALDO_MEDIO_CONSUMO: 0
Columna SALDO_MEDIO_TARJETAS: 0
Columna SALDO_MEDIO_HIPOTECARIO: 0
Columna SALDO_MEDIO_CARTERA: 0
Columna SALDO_MEDIO_LEASING: 0
Columna SALDO_MEDIO_PRESTAMOS_COMERCIALES: 0
Columna SALDO_MEDIO_COMEXT: 0
Columna SALDO_MEDIO_TJ_EMPRESAS: 0
Columna T_NETCASH: 0
Columna T_SEGURO: 0
Columna T_WALLET: 0
Columna T_TCAMBIO: 0
Columna T_AFILIACION_DIGITAL: 0
Columna T_CARGO_RECURRENTE: 0
Columna SALDO_MEDIO_INVERSION_RENTABLE: 0


In [21]:
# Exportar a repositorio
df_bbva[df_bbva.GENERO == 'X'].to_csv(HT_PATH + '/Tablas_guardadas/bbva_morales.csv')
df_bbva[df_bbva.GENERO != 'X'].to_csv(HT_PATH + '/Tablas_guardadas/bbva_fisicas.csv')

# Exportar a repositorio
df_otros[df_otros.GENERO == 'X'].to_csv(HT_PATH + '/Tablas_guardadas/otros_morales.csv')
df_otros[df_otros.GENERO != 'X'].to_csv(HT_PATH + '/Tablas_guardadas/otros_fisicas.csv')